<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/ENDS9_TF_OpenBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jan  7 14:55:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd 
import json
import random
import math
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Then set a random seed for deterministic results/reproducability.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!pip install jsonlines
import jsonlines

In [ ]:
sentence = []
label = []
with open('/content/OpenBookQa/train.jsonl') as h:
    for line in h:
        example = json.loads(line)
        print(example.keys())
        # scores = []
        merged_choices = ' A: '.join([choice['text'] for choice in example['question']['choices']])
        input = 'Q: ' + example['question']['stem'] + ' A: ' + merged_choices
        correct_answer = [ choice['text'] for choice in example['question']['choices'] if choice['label'] == example['answerKey'] ][0]
        sentence.append(input)
        label.append(correct_answer)
        #print(input, correct_answer)
dataset_df = pd.DataFrame({'sentence':sentence, 'label':label})

In [ ]:
dataset_df

,sentence,label
0,Q: The sun is responsible for A: puppies learn...,"plants sprouting, blooming and wilting"
1,Q: When standing miles away from Mount Rushmor...,the mountains seem smaller than in photographs
2,Q: When food is reduced in the stomach A: the ...,nutrients are being deconstructed
3,Q: Stars are A: warm lights that float A: made...,great balls of gas burning billions of miles away
4,Q: You can make a telescope with a A: straw A:...,mailing tube
...,...,...
3409,Q: Bears instinctively know when its time to h...,Their parents
3410,Q: An example of a fossil is a paw print in wh...,hard stones
3411,Q: Which is likeliest to make light pass throu...,any kind of tangible object
3412,Q: What most likely caused the castle built by...,very strong eddies of fast moving air carried ...


Instantiate our German and English spaCy models.

In [ ]:
!python3 -m spacy download de_core_news_sm

     |████████████████████████████████| 14.9MB 9.7MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=bef19d049a4a328fb8aa383e8d700b51dab3d254d44552ab104bb67962df8744
  Stored in directory: /tmp/pip-ephem-wheel-cache-29sy714y/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [ ]:
import de_core_news_sm

In [ ]:
spacy_de =  de_core_news_sm.load()
spacy_en = spacy.load('en')

Previously we reversed the source (German) sentence, however in the paper we are implementing they don't do this, so neither will we.

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens

In [ ]:
from torchtext import data 
import matplotlib.pyplot as plt
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =False, include_lengths=False, lower=False)
Label = data.Field(sequential =True, tokenize ='spacy', is_target=False, batch_first =False, include_lengths=False, lower=False)

In [ ]:
dataset = dataset_df
fields = [('sentence', Sentence),('label',Label)]
example = [data.Example.fromlist([dataset.sentence[i],dataset.label[i]], fields) for i in range(dataset.shape[0])] 
commonqa_ds = data.Dataset(example, fields)
(train, valid) = commonqa_ds.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [ ]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  9462
Size of label vocab :  3610
Top 10 words appreared repeatedly : [(')', 55314), ('(', 55313), ('A', 7331), ('H', 6944), ('C', 6923), ('B', 6921), ('D', 6916), ('\\n', 6914), ('E', 6914), ('F', 6914)]
Labels :  defaultdict(<function _default_unk_index at 0x7f6c2fa93f28>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'of': 3, 'a': 4, 'and': 5, 'energy': 6, 'water': 7, '.': 8, 'to': 9, 'heat': 10, 'in': 11, 'plants': 12, 'it': 13, 'food': 14, 'animals': 15, 'cells': 16, 'light': 17, 'humans': 18, 'soil': 19, 'air': 20, 'body': 21, 'The': 22, 'A': 23, 'bacteria': 24, "'s": 25, 'Something': 26, 'chemical': 27, 'them': 28, ',': 29, 'an': 30, 'rain': 31, 'wind': 32, '-': 33, 'oxygen': 34, 'It': 35, 'insects': 36, 'on': 37, 'sunlight': 38, 'blood': 39, 'life': 40, 'their': 41, 'system': 42, 'organisms': 43, 'reproduction': 44, 'sun': 45, 'electricity': 46, 'is': 47, 'temperature': 48, 'acid': 49, 'viruses': 50, 'skin': 51, 'trees': 52, 'cancer': 53, 'from': 54, 'pressure': 55, 

In [ ]:
Sentence.vocab.stoi['new construction'], Label.vocab.stoi['new construction'] 

(0, 0)

Load our data.

We'll also print out an example just to double check they're not reversed.

Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

Finally, define the `device` and create our iterators.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE=128
train_iterator, test_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False, shuffle=False)

In [ ]:
iter_one = next(iter(train_iterator))

In [ ]:
iter_one.label.shape

torch.Size([14, 128])

In [ ]:
for entry in (iter_one.sentence[0][0,:]):
    print(entry.item())

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

## Model Checks

In [ ]:
next(iter(train_iterator_de_en)), iter_one

(
 [torchtext.data.batch.Batch of size 128 from MULTI30K]
 	[.src]:[torch.LongTensor of size 34x128]
 	[.trg]:[torch.LongTensor of size 35x128], 
 [torchtext.data.batch.Batch of size 128]
 	[.sentence]:('[torch.LongTensor of size 128x64]', '[torch.LongTensor of size 128]')
 	[.label]:('[torch.LongTensor of size 128x4]', '[torch.LongTensor of size 128]'))

In [ ]:
iter_one.sentence[1],iter_one.sentence[0][1]

NameError: ignored

In [ ]:
text, text_lengths = iter_one.sentence
embedded = nn.Embedding(len(Sentence.vocab), 256)(text)
packed_seq = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True, enforce_sorted=False) 
outputs, hidden = nn.GRU(256, 512)(packed_seq)

In [ ]:
outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
hidden, hidden_lengths = torch.nn.utils.rnn.pad_packed_sequence(hidden)

In [ ]:
hidden, outputs

In [ ]:
hidden.shape,dec_embed.shape

(torch.Size([1, 128, 512]), torch.Size([1, 128, 256]))

In [ ]:
label_text, label_text_lengths = iter_one.label
label_embedded = nn.Embedding(len(Label.vocab), 256)(label_text)
label_packed_seq = nn.utils.rnn.pack_padded_sequence(label_embedded, label_text_lengths.cpu(), batch_first=False, enforce_sorted=False) 
label_outputs, label_hidden = nn.GRU(256, 512)(label_packed_seq)

In [ ]:
embedded.shape, text.shape
non_padded_output = nn.GRU(256, 512)(embedded)


In [ ]:
print(non_padded_output[0].shape, non_padded_output[1].shape, outputs.data.shape, hidden.shape)
print(label_outputs.data.shape, label_hidden.data.shape)

torch.Size([128, 64, 512]) torch.Size([1, 64, 512]) torch.Size([4986, 512]) torch.Size([1, 128, 512])
torch.Size([207, 512]) torch.Size([1, 128, 512])


In [ ]:
len(Sentence.vocab),len(Label.vocab)

(4344, 986)

In [ ]:
iter_de_en = next(iter(train_iterator_de_en))
embedded = nn.Embedding(len(SRC.vocab), 256)(iter_de_en.src)

output_deen, hidden_deen = nn.GRU(256, 512)(embedded)

In [ ]:
hidden_deen.shape, hidden.shape

(torch.Size([1, 128, 512]), torch.Size([1, 128, 512]))

In [ ]:
label_text, label_text_lengths =iter_one.label

In [ ]:
print(label_text.shape, label_text_lengths.shape, iter_de_en.trg.shape)
label_packed_seq = nn.utils.rnn.pack_padded_sequence(label_text, label_text_lengths.cpu(), batch_first=True, enforce_sorted=False) 
label_packed_seq.sorted_indices.shape

torch.Size([128, 4]) torch.Size([128]) torch.Size([31, 128])


torch.Size([128])

In [ ]:
label_text.shape

torch.Size([128, 4])

In [ ]:
label_packed_seq.data.shape

torch.Size([207])

In [ ]:
iter_de_en.src.shape, iter_de_en.trg.shape

(torch.Size([36, 128]), torch.Size([31, 128]))

In [ ]:
label_text_reshaped =  label_text.reshape(4, -1)

In [ ]:
input_to_decoder.shape

torch.Size([128])

In [ ]:
# input_to_decoder = label_text_reshaped[0,:]
# input_to_decoder = input_to_decoder.unsqueeze(0)

# INPUT_DIM = len(Sentence.vocab)
# OUTPUT_DIM = len(Label.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# HID_DIM = 512
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

# enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
# dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)


dec_embed = nn.Embedding(len(Label.vocab), 256)(input_to_decoder)

In [ ]:
dec_embed.shape
dec_output, dec_hidden = nn.GRU(256, 512)(dec_embed)

In [ ]:
dec_embed.squeeze(0).shape,hidden.shape

(torch.Size([128, 256]), torch.Size([1, 128, 512]))

In [ ]:
torch.cat((dec_embed, hidden), dim=2).shape

torch.Size([1, 128, 768])

In [ ]:
dec_output.shape, dec_hidden.shape, hidden.shape

(torch.Size([1, 128, 512]),
 torch.Size([1, 128, 512]),
 torch.Size([1, 128, 512]))

In [ ]:
for entry in label_packed_seq.data:
    print(Label.vocab.itos[entry.item()])

In [ ]:
iter_de_en.trg[0,:].shape, iter_one.label[0]

In [ ]:
label_text.shape

torch.Size([128, 4])

In [ ]:
label_text_reshaped[0]

tensor([196,  36,   3,   1, 393,   1,   1,   1,  35,  82,   1,   1, 971,   1,
          1,   1, 888,   1,   1,   1,  28,   1,   1,   1,  17, 300, 551,   1,
        791,  21,   1,   1, 605,   1,   1,   1, 215, 304,   1,   1,  58,  11,
          5,   1, 428,  52,   1,   1, 108,  64,   1,   1, 705,   1,   1,   1,
        549,   1,   1,   1,  51,   3,  16,   1, 148, 701,   1,   1,   6,   1,
          1,   1,  44,   1,   1,   1,  12,   1,   1,   1, 410,   1,   1,   1,
        795, 772,   1,   1, 688,   1,   1,   1, 611,   1,   1,   1, 592, 719,
          1,   1, 188,   1,   1,   1,  34, 200, 497,   1, 563, 857,   1,   1,
        601,   1,   1,   1, 771, 951,   1,   1, 665, 974,   1,   1,  25,   1,
          1,   1])

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        text, text_lengths = src
        embedded = self.dropout(self.embedding(text))
        # packed_seq = nn.utils.rnn.pack_padded_sequence(embedded, 
        #                                                text_lengths.cpu(),
        #                                                batch_first=False,
        #                                                enforce_sorted=False)        
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state! 
        ## outputs is a packed sequence but since it is not used we will not 
        ## unpack it         
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Model Definition

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        # print(embedded.shape, hidden.shape, context.shape)
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)

INPUT_DIM = len(Sentence.vocab)
OUTPUT_DIM = len(Label.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 256
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8645, 256)
    (rnn): GRU(256, 256)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3482, 256)
    (rnn): GRU(512, 256)
    (fc_out): Linear(in_features=768, out_features=3482, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,768,282 trainable parameters


We initiaize our optimizer.

In [ ]:
from torch.optim.lr_scheduler import StepLR, OneCycleLR, MultiStepLR, CyclicLR, ReduceLROnPlateau

optimizer = optim.Adam(model.parameters(), lr=0.0009893272, weight_decay=0.0001)
scheduler = OneCycleLR(optimizer, 
                        0.001, 
                        epochs=100, 
                        cycle_momentum=False, 
                        steps_per_epoch=len(train_iterator), 
                        #base_momentum=config.momentum,
                        #max_momentum=0.95, 
                        pct_start=0.208,
                        # anneal_strategy=config.anneal_strategy,
                        div_factor=100,
                        # final_div_factor=config.final_div_factor
                        )
# 0.000981989011942079

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [ ]:
#TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

TRG_PAD_IDX = Label.vocab.stoi[Label.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.sentence
        trg = batch.label
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        # print("output_dim:", output.shape)
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        # print("output_dim before loss:", output.shape, trg.shape)
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        # scheduler.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
iter_one.label[0].shape

torch.Size([128, 4])

In [ ]:
iter_one = next(iter(train_iterator))
iter_one.label[0][1:].view(-1)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # src = batch.src
            # trg = batch.trg
            src = batch.sentence
            trg = batch.label
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 1e-06
    lr: 9.999999999999159e-07
    max_lr: 0.001
    min_lr: 9.999999999999999e-11
    weight_decay: 0.0001
)

Then, we train our model, saving the parameters that give us the best validation loss.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.001)

In [ ]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')
# model.load_state_dict(torch.load('tut2-model.pt'))
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | LR: {scheduler.get_last_lr()}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 1s
	Train Loss: 8.154 | Train PPL: 3478.227 | LR: [1.0000000000000026e-05]
	 Val. Loss: 8.150 |  Val. PPL: 3462.884
Epoch: 02 | Time: 0m 1s
	Train Loss: 8.154 | Train PPL: 3475.632 | LR: [1.0000000000000026e-05]
	 Val. Loss: 8.149 |  Val. PPL: 3460.940
Epoch: 03 | Time: 0m 1s
	Train Loss: 8.153 | Train PPL: 3472.959 | LR: [1.0000000000000026e-05]
	 Val. Loss: 8.149 |  Val. PPL: 3458.951
Epoch: 04 | Time: 0m 1s
	Train Loss: 8.152 | Train PPL: 3470.415 | LR: [1.0000000000000026e-05]
	 Val. Loss: 8.148 |  Val. PPL: 3456.900
Epoch: 05 | Time: 0m 1s
	Train Loss: 8.151 | Train PPL: 3467.501 | LR: [1.0000000000000026e-05]
	 Val. Loss: 8.147 |  Val. PPL: 3454.503
Epoch: 06 | Time: 0m 1s
	Train Loss: 8.150 | Train PPL: 3464.282 | LR: [1.0000000000000026e-05]
	 Val. Loss: 8.147 |  Val. PPL: 3451.783
Epoch: 07 | Time: 0m 1s
	Train Loss: 8.149 | Train PPL: 3460.747 | LR: [1.0000000000000026e-05]
	 Val. Loss: 8.146 |  Val. PPL: 3448.774
Epoch: 08 | Time: 0m 1s
	Train Loss: 8.14

KeyboardInterrupt: ignored

In [ ]:
best_valid_loss

7.396448850631714

In [ ]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = 7.396448850631714
model.load_state_dict(torch.load('tut2-model.pt'))

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 1s
	Train Loss: 6.949 | Train PPL: 1041.755
	 Val. Loss: 7.394 |  Val. PPL: 1626.104
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.946 | Train PPL: 1038.512
	 Val. Loss: 7.392 |  Val. PPL: 1623.474
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.943 | Train PPL: 1035.957
	 Val. Loss: 7.391 |  Val. PPL: 1620.960
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.941 | Train PPL: 1033.612
	 Val. Loss: 7.389 |  Val. PPL: 1618.544
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.939 | Train PPL: 1031.422
	 Val. Loss: 7.388 |  Val. PPL: 1616.215
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.937 | Train PPL: 1029.345
	 Val. Loss: 7.386 |  Val. PPL: 1613.964
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.935 | Train PPL: 1027.347
	 Val. Loss: 7.385 |  Val. PPL: 1611.787
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.933 | Train PPL: 1025.482
	 Val. Loss: 7.384 |  Val. PPL: 1609.675
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.931 | Train PPL: 1023.749
	 Val. Loss: 7.383 |  Val. PPL: 1607.624
Epoch: 10 | Time: 0m 1s
	Train Loss: 6.930 | T

In [ ]:
print(best_valid_loss)

7.326877593994141


In [ ]:
N_EPOCHS = 50
CLIP = 1
print(best_valid_loss)
model.load_state_dict(torch.load('tut2-model.pt'))

best_valid_loss = 7.326877593994141 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

7.326877593994141
Epoch: 01 | Time: 0m 1s
	Train Loss: 6.890 | Train PPL: 982.353
	 Val. Loss: 7.327 |  Val. PPL: 1520.165
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.890 | Train PPL: 982.263
	 Val. Loss: 7.326 |  Val. PPL: 1519.710
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.890 | Train PPL: 982.148
	 Val. Loss: 7.326 |  Val. PPL: 1519.261
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.890 | Train PPL: 982.069
	 Val. Loss: 7.326 |  Val. PPL: 1518.820
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.890 | Train PPL: 981.943
	 Val. Loss: 7.325 |  Val. PPL: 1518.384
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.889 | Train PPL: 981.854
	 Val. Loss: 7.325 |  Val. PPL: 1517.954
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.889 | Train PPL: 981.782
	 Val. Loss: 7.325 |  Val. PPL: 1517.529
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.889 | Train PPL: 981.647
	 Val. Loss: 7.325 |  Val. PPL: 1517.110
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.889 | Train PPL: 981.560
	 Val. Loss: 7.324 |  Val. PPL: 1516.697
Epoch: 10 | Time: 0m 1s
	Train Loss: 

In [ ]:
print(best_valid_loss)

7.315857172012329


In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-06
    weight_decay: 0.1
)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-06, weight_decay=0.1)
# scheduler = OneCycleLR(optimizer, 
#                         config.ocp_max_lr, 
#                         epochs=config.epochs, 
#                         cycle_momentum=cycle_momentum, 
#                         steps_per_epoch=len(trainloader), 
#                         base_momentum=config.momentum,
#                         max_momentum=0.95, 
#                         pct_start=0.208,
#                         anneal_strategy=config.anneal_strategy,
#                         div_factor=config.div_factor,
#                         final_div_factor=config.final_div_factor
#                         )

In [ ]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = 7.315857172012329 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 1s
	Train Loss: 6.886 | Train PPL: 978.660
	 Val. Loss: 7.314 |  Val. PPL: 1501.789
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.887 | Train PPL: 979.204
	 Val. Loss: 7.313 |  Val. PPL: 1499.645
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.888 | Train PPL: 979.963
	 Val. Loss: 7.312 |  Val. PPL: 1497.530
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.888 | Train PPL: 980.739
	 Val. Loss: 7.310 |  Val. PPL: 1495.444
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.889 | Train PPL: 981.437
	 Val. Loss: 7.309 |  Val. PPL: 1493.384
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.890 | Train PPL: 982.210
	 Val. Loss: 7.307 |  Val. PPL: 1491.352
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.891 | Train PPL: 982.978
	 Val. Loss: 7.306 |  Val. PPL: 1489.343
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.891 | Train PPL: 983.760
	 Val. Loss: 7.305 |  Val. PPL: 1487.360
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.892 | Train PPL: 984.524
	 Val. Loss: 7.303 |  Val. PPL: 1485.400
Epoch: 10 | Time: 0m 1s
	Train Loss: 6.893 | Train PPL:

In [ ]:
best_valid_loss

7.259694695472717

In [ ]:
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('tut2-model.pt'))
best_valid_loss = 7.259694695472717 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 1s
	Train Loss: 6.928 | Train PPL: 1020.265
	 Val. Loss: 7.259 |  Val. PPL: 1420.590
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.929 | Train PPL: 1021.090
	 Val. Loss: 7.258 |  Val. PPL: 1419.370
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.929 | Train PPL: 1021.905
	 Val. Loss: 7.257 |  Val. PPL: 1418.161
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.930 | Train PPL: 1022.741
	 Val. Loss: 7.256 |  Val. PPL: 1416.960
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.931 | Train PPL: 1023.542
	 Val. Loss: 7.255 |  Val. PPL: 1415.773
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.932 | Train PPL: 1024.350
	 Val. Loss: 7.255 |  Val. PPL: 1414.595
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.933 | Train PPL: 1025.141
	 Val. Loss: 7.254 |  Val. PPL: 1413.426
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.933 | Train PPL: 1025.946
	 Val. Loss: 7.253 |  Val. PPL: 1412.270
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.934 | Train PPL: 1026.753
	 Val. Loss: 7.252 |  Val. PPL: 1411.120
Epoch: 10 | Time: 0m 1s
	Train Loss: 6.935 | T

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=5e-8, weight_decay=0.005)

In [ ]:
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('tut2-model.pt'))
best_valid_loss = 7.211369276046753 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 1s
	Train Loss: 6.973 | Train PPL: 1067.101
	 Val. Loss: 7.211 |  Val. PPL: 1354.013
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.973 | Train PPL: 1067.649
	 Val. Loss: 7.210 |  Val. PPL: 1353.283
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.974 | Train PPL: 1068.173
	 Val. Loss: 7.210 |  Val. PPL: 1352.560
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.974 | Train PPL: 1068.695
	 Val. Loss: 7.209 |  Val. PPL: 1351.843
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.975 | Train PPL: 1069.220
	 Val. Loss: 7.209 |  Val. PPL: 1351.131
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.975 | Train PPL: 1069.757
	 Val. Loss: 7.208 |  Val. PPL: 1350.424
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.976 | Train PPL: 1070.293
	 Val. Loss: 7.208 |  Val. PPL: 1349.724
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.976 | Train PPL: 1070.809
	 Val. Loss: 7.207 |  Val. PPL: 1349.027
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.977 | Train PPL: 1071.332
	 Val. Loss: 7.207 |  Val. PPL: 1348.338
Epoch: 10 | Time: 0m 1s
	Train Loss: 6.977 | T

In [ ]:
N_EPOCHS = 100
CLIP = 1
model.load_state_dict(torch.load('/content/tut2-model.pt'))
best_valid_loss = 7.188767671585083 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 1s
	Train Loss: 6.995 | Train PPL: 1090.953
	 Val. Loss: 7.188 |  Val. PPL: 1323.990
Epoch: 02 | Time: 0m 1s
	Train Loss: 6.995 | Train PPL: 1091.381
	 Val. Loss: 7.188 |  Val. PPL: 1323.515
Epoch: 03 | Time: 0m 1s
	Train Loss: 6.996 | Train PPL: 1091.789
	 Val. Loss: 7.188 |  Val. PPL: 1323.043
Epoch: 04 | Time: 0m 1s
	Train Loss: 6.996 | Train PPL: 1092.202
	 Val. Loss: 7.187 |  Val. PPL: 1322.578
Epoch: 05 | Time: 0m 1s
	Train Loss: 6.996 | Train PPL: 1092.605
	 Val. Loss: 7.187 |  Val. PPL: 1322.116
Epoch: 06 | Time: 0m 1s
	Train Loss: 6.997 | Train PPL: 1093.027
	 Val. Loss: 7.187 |  Val. PPL: 1321.659
Epoch: 07 | Time: 0m 1s
	Train Loss: 6.997 | Train PPL: 1093.425
	 Val. Loss: 7.186 |  Val. PPL: 1321.206
Epoch: 08 | Time: 0m 1s
	Train Loss: 6.997 | Train PPL: 1093.830
	 Val. Loss: 7.186 |  Val. PPL: 1320.757
Epoch: 09 | Time: 0m 1s
	Train Loss: 6.998 | Train PPL: 1094.228
	 Val. Loss: 7.186 |  Val. PPL: 1320.313
Epoch: 10 | Time: 0m 1s
	Train Loss: 6.998 | T

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-05, weight_decay=0.1)

In [ ]:
N_EPOCHS = 50
CLIP = 1
model.load_state_dict(torch.load('/content/tut2-model.pt'))
best_valid_loss = 7.150131702423096 #float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
print(best_valid_loss)

Epoch: 01 | Time: 0m 1s
	Train Loss: 7.049 | Train PPL: 1152.105
	 Val. Loss: 7.149 |  Val. PPL: 1272.257
Epoch: 02 | Time: 0m 1s
	Train Loss: 7.044 | Train PPL: 1145.868
	 Val. Loss: 7.148 |  Val. PPL: 1271.496
Epoch: 03 | Time: 0m 1s
	Train Loss: 7.044 | Train PPL: 1145.672
	 Val. Loss: 7.148 |  Val. PPL: 1272.085
Epoch: 04 | Time: 0m 1s
	Train Loss: 7.043 | Train PPL: 1145.171
	 Val. Loss: 7.149 |  Val. PPL: 1272.417
Epoch: 05 | Time: 0m 1s
	Train Loss: 7.043 | Train PPL: 1144.855
	 Val. Loss: 7.149 |  Val. PPL: 1272.848
Epoch: 06 | Time: 0m 1s
	Train Loss: 7.043 | Train PPL: 1144.437
	 Val. Loss: 7.149 |  Val. PPL: 1273.244
Epoch: 07 | Time: 0m 1s
	Train Loss: 7.042 | Train PPL: 1144.029
	 Val. Loss: 7.150 |  Val. PPL: 1273.638
Epoch: 08 | Time: 0m 1s
	Train Loss: 7.042 | Train PPL: 1143.605
	 Val. Loss: 7.150 |  Val. PPL: 1274.014
Epoch: 09 | Time: 0m 1s
	Train Loss: 7.042 | Train PPL: 1143.176
	 Val. Loss: 7.150 |  Val. PPL: 1274.373
Epoch: 10 | Time: 0m 1s
	Train Loss: 7.041 | T

In [ ]:
iter_one = next(iter(train_iterator))
iter_one.label.shape

torch.Size([9, 128])

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 10.525 | Test PPL: 37253.098 |


Just looking at the test loss, we get better performance. This is a pretty good sign that this model architecture is doing something right! Relieving the information compression seems like the way forard, and in the next tutorial we'll expand on this even further with *attention*.